# Preprocess for Language Model

수집한 Corpus를 통합하여 일련의 규칙으로 전처리

* 필요한 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import random
import pickle

* 수집한 corpus 통합

In [2]:
kor_corpus = pd.read_csv('../data/OpenSubtitles.csv', encoding='utf-8')
sub_corpus = pd.read_csv('../data/new_kor.csv', encoding='cp949')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

sub_corpus = pd.read_csv('../data/tatoeba.csv', encoding='utf-8')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

sub_corpus = pd.read_csv('../data/네이버 해커톤 데이터셋.csv', encoding='utf-8', engine='python')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

for i in range(10):
    sub_corpus = pd.read_csv('../data/AI Hub 한-영 데이터셋[%s].csv' % i, encoding='utf-8', engine='python')
    kor_corpus = pd.concat([kor_corpus, sub_corpus])

* corpus 크기 확인

In [3]:
len(kor_corpus)

3048264

* 코퍼스 내용 확인

In [4]:
kor_corpus.head()

,ko
0,"폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,황새 아저씨를 기다리세요
3,찾아와 선물을 주실 거예요
4,가난하든 부자이든 상관이 없답니다


* list로 변환 및 shuffle (기존에는 그냥 concatenate 했기 때문에 섞어줌)

In [5]:
kor_corpus = list(kor_corpus['ko'])
random.shuffle(kor_corpus)
kor_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아!',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네, 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '-뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스, 토니 그란데, 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후 (현지시간) 오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은 “(증인신문 내용이) 다 녹음 됐으니까 (이 전 대통령이 한 말에 대해) 따지려면 따져볼 수 있다”고 말하기도 했다.']

* ( ) 와 같이 괄호 처리된 내용들을 필터링한다. (Speech-To-Text에 사용할 언어모델이기 때문에 괄호 및 괄호 내용은 필요없음)

In [6]:
new_corpus = list()
flag = False

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch == '(':
            flag = True
            continue
        elif ch == ')':
            flag = False
            continue
        elif flag == False:
            new_sentence += ch
        else:
            continue
            
    new_corpus.append(new_sentence)

* ()가 잘 필터링 됐는지 확인

In [7]:
kor_corpus = new_corpus
kor_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아!',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네, 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '-뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스, 토니 그란데, 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후  오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은 “ 다 녹음 됐으니까  따지려면 따져볼 수 있다”고 말하기도 했다.']

* 현재 corpus에 어떤 특수문자들이 있는지 확인

In [8]:
special_ch = list()

for sentence in kor_corpus:
    for ch in sentence:
        if ch.isdigit() == False and ch.isalpha() == False and ch not in special_ch:
            special_ch.append(ch)

* corpus에 등장한 특수문자 개수 확인

In [9]:
len(special_ch)

209

* 등장한 특수문자를 csv 파일로 저장하여 적절한 발음전사로 매핑

In [10]:
special_df = pd.read_csv('special.csv', encoding='utf-8')
special_df = special_df.fillna('')
special_df.head()

,special,replace
0,,
1,.,.
2,“,
3,"""",
4,-,


* special_dict 생성 (special => replace로 변환)

In [11]:
special_dict = dict()

for (special, replace) in zip(special_df['special'], special_df['replace']):
    special_dict[special] = replace

special_dict[' '] = ' '

* in 연산자로 체크를 위해 list 형식으로 저장

In [12]:
specials = list(special_df['special'])
replaces = list(special_df['replace'])

* 특수문자들을 원하는 발음전사로 변환

In [13]:
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch in specials:
            new_sentence += special_dict[ch]
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

* 특수문자가 적절히 변경되었는지 확인

In [14]:
new_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아!',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스 토니 그란데 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후  오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은  다 녹음 됐으니까  따지려면 따져볼 수 있다고 말하기도 했다.']

* corpus 업데이트

In [15]:
kor_corpus = new_corpus

* 알파벳 필터링 (소문자)

In [16]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch in alphabet:
            continue
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

In [17]:
kor_corpus = new_corpus
kor_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아!',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스 토니 그란데 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후  오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은  다 녹음 됐으니까  따지려면 따져볼 수 있다고 말하기도 했다.']

* 대문자 필터링 (발음으로 전사 ex) K리그 => 케이리그)

In [18]:
new_corpus = list()
upper = ['A', 'B', 'C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
upper_dict = {'A' : '에이', 'B' : '비', 'C' : '씨','D' : '디','E' : '이','F' : '에프','G' : '쥐',
              'H' : '에이취','I' : '아이','J' : '제이','K' : '케이','L' : '엘','M' : '엠','N' : '엔',
              'O' : '오','P' : '피','Q' : '큐','R' : '알','S' : '에스','T' : '티','U' : '유','V' : '브이','W' : '떠블유',
              'X' : '엑스','Y' : '와이','Z' : '지'}

for sentence in kor_corpus:
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch in upper:
            if idx == 0 and (idx == len(sentence) - 1 or sentence[idx + 1] == ' '):
                continue
                
            elif idx != 0 and idx < len(sentence) - 1 and sentence[idx + 1] == ' ' and sentence[idx - 1] == ' ':
                continue
                
            elif idx != 0 and sentence[idx -1] == ' ' and idx + 1 == len(sentence):
                continue
                
            else:
                new_sentence += upper_dict[ch]
            
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

In [19]:
new_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아!',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스 토니 그란데 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후  오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은  다 녹음 됐으니까  따지려면 따져볼 수 있다고 말하기도 했다.']

* 데이터셋 업데이트

In [20]:
kor_corpus = new_corpus

* 긴 공백 => ' '

In [21]:
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    tokens = sentence.split()
    for idx, token in enumerate(tokens):
        if idx == 0:
            new_sentence += token + ' '
        elif idx == len(tokens) - 1:
            new_sentence += token
        else:
            new_sentence += token + ' '
            
    new_corpus.append(new_sentence)

In [22]:
new_corpus[:20]

['어디 보자...',
 '7대 왕국 종족 중에 오크는 처음 듣는다',
 '이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.',
 '라니스터 가의',
 '별 희한한 생각이 다 떠오르곤 하죠',
 '아! ',
 '회의 전에 마실 것 좀 드릴까요? 커피와 차가 있습니다.',
 '나는 그런 일에는 전혀 흥미가 없어요.',
 '일자리가 있고 제대로 된 회사가 있다면 늘 사람들이 붐비고 활력이 넘치는 도시로 자리매김할 수 있다.',
 '카카오가 많이 함유된 다크 초콜릿이 더 좋다는 것을 잊지 말자.',
 '비오 13세는 누구인가?',
 '네 어..',
 '치밀한 사람이긴 하지만...',
 '계속 절 따라오셨죠',
 '세계 4위 특허출원 강국이지만 지식재산 심사 품질과 보호 수준이 낮아 가치를 제대로 인정받지 못하고 있다.',
 '나는 전부터 이 학교를 잘 알고 있었어.',
 '뭔가 꿍꿍이를 꾸미는게 틀림없어',
 '어쩔 수 없이 과학 수업에 늦었습니다.',
 '가르시아 에르난데스 토니 그란데 하비에르 미냐노 등 축구대표팀의 외국인 코치들이 8일 오후 오스트리아 레오강의 대표팀 숙소에서 인터뷰에 응하고있다.',
 '검찰은 다 녹음 됐으니까 따지려면 따져볼 수 있다고 말하기도 했다.']

* 데이터셋 업데이트

In [23]:
kor_corpus = new_corpus

* 데이터셋 크기 확인

In [24]:
len(kor_corpus)

3048264

* 필터링 되면서 빈문자열만 남은 데이터 제외

In [25]:
new_corpus = list()

for sentence in kor_corpus:
    if len(sentence) == 0:
        continue
    new_corpus.append(sentence)

* 약 25,000개 필터링

In [26]:
len(new_corpus)

3023296

* 데이터셋 업데이트

In [27]:
kor_corpus = new_corpus

* 아무지점이나 찍어서 중간확인

In [28]:
kor_corpus[6000:6020]

['이란 세계식량계획 사무소 대표 및 국가 담당국장 쥐는 2018년 올해 주이란 한국대사관 측이 220만 달러를 기부했을 때 한국 국민과 정부에 감사의 뜻을 전하기도 했다.',
 '조심해요 오염된 거예요',
 '그건 그렇고 메이즈는 어때?',
 '간에 있는 덩어리가..',
 '. ',
 '물렸어요? ',
 '안 보이나 보네',
 '그 전엔 아무것도 쓸 것 이 없었어요. 닥터 그레이.',
 '하루만 머무를 거요',
 '그런건 내 일이 아니라서 다른 도움이 필요하면 말해 난 쇼파에 있을꺼야',
 '고마워 ',
 '취업준비생들은 현실적으로 청년취업이 어려운 상황에서 아쉽다는 반응이었다.',
 '시장을 떠나는 순간 덮치겠음',
 '그리고 디자인이랑',
 '저항권이란 위헌적인 국가권력의 행사 때문에 헌법이 침해 또는 파괴되는 경우 주권자인 국민이 헌법을 보장하기 위해 최후 비상수단으로 국가권력에 실력으로 저항하는 것입니다.',
 '현대모비스 양동근도 정규 리그에서 상대해 본 결과 함지훈을 막을 선수가 없다는 게 전자랜드의 약점이라고 지적했다.',
 '아사히는 꾸준히 한국어 공부를 하면서 내가 하고 싶은 노래를 만들기 위해 항상 들고 다닌다고 말해 아티스트로서 더욱 발전할 수 있는 가능성을 보여줬다.',
 '서양마사지 방식으로는 스웨디시 마사지 에스알엔 그리고 홀리스틱 마사지가 있다.',
 '내 말을 들었더라면',
 '앞으로 이렇게 수도권 광역 경제발전위원회가 자주 열려서 다양한 이야기를 해야 한다.']

* 뒤에가 공백으로 끝나는 문자열 공백 제거

In [29]:
for idx, sentence in enumerate(kor_corpus):
    if sentence[-1] == ' ':
        kor_corpus[idx] = sentence[:-1] 

In [30]:
kor_corpus[6000:6020]

['이란 세계식량계획 사무소 대표 및 국가 담당국장 쥐는 2018년 올해 주이란 한국대사관 측이 220만 달러를 기부했을 때 한국 국민과 정부에 감사의 뜻을 전하기도 했다.',
 '조심해요 오염된 거예요',
 '그건 그렇고 메이즈는 어때?',
 '간에 있는 덩어리가..',
 '.',
 '물렸어요?',
 '안 보이나 보네',
 '그 전엔 아무것도 쓸 것 이 없었어요. 닥터 그레이.',
 '하루만 머무를 거요',
 '그런건 내 일이 아니라서 다른 도움이 필요하면 말해 난 쇼파에 있을꺼야',
 '고마워',
 '취업준비생들은 현실적으로 청년취업이 어려운 상황에서 아쉽다는 반응이었다.',
 '시장을 떠나는 순간 덮치겠음',
 '그리고 디자인이랑',
 '저항권이란 위헌적인 국가권력의 행사 때문에 헌법이 침해 또는 파괴되는 경우 주권자인 국민이 헌법을 보장하기 위해 최후 비상수단으로 국가권력에 실력으로 저항하는 것입니다.',
 '현대모비스 양동근도 정규 리그에서 상대해 본 결과 함지훈을 막을 선수가 없다는 게 전자랜드의 약점이라고 지적했다.',
 '아사히는 꾸준히 한국어 공부를 하면서 내가 하고 싶은 노래를 만들기 위해 항상 들고 다닌다고 말해 아티스트로서 더욱 발전할 수 있는 가능성을 보여줬다.',
 '서양마사지 방식으로는 스웨디시 마사지 에스알엔 그리고 홀리스틱 마사지가 있다.',
 '내 말을 들었더라면',
 '앞으로 이렇게 수도권 광역 경제발전위원회가 자주 열려서 다양한 이야기를 해야 한다.']

* 현재 데이터셋 크기 확인

In [31]:
len(kor_corpus)

3023296

* '?', '!', '.'만 남은 문자열 필터링

In [32]:
new_corpus = list()

for sentence in kor_corpus:
    if sentence == '?' or sentence == '!' or sentence == '.':
        continue
        
    else:
        new_corpus.append(sentence)
        
len(new_corpus)

3001447

* 약 2만개 필터링 후 데이터셋 업데이트

In [33]:
kor_corpus = new_corpus

* import **KoNPron** (Korean Number Pronunciaion) => 숫자를 발음전사로 변환

In [34]:
from konpron import KoNPron

kpr = KoNPron()

* **KoNPron** 테스트

In [35]:
sentence = "1234랑, 12.34랑, 1,234랑, 123^4랑, 1²³⁴랑, 1.23^4랑, 12.3⁴랑, 12·34랑, 293301300.55301⁴"

ret = kpr.convert(sentence)
if ret is not None:
    print(ret)

천이백삼십사랑, 십이 점 삼사랑, 천이백삼십사랑, 백이십삼의 사 승랑, 일의 이백삼십사 승랑, 일 점 이삼의 사 승랑, 십이 점 삼의 사 승랑, 일이삼사랑, 이억 구천삼백삼십만 천삼백 점 오오삼영일의 사 승


* **KoNPron**으로 숫자 => 발음전사로 변환

In [36]:
new_corpus = list()

for sentence in kor_corpus:
    try:
        ret = kpr.convert(sentence)
        if ret is not None:
            new_corpus.append(ret)
        else:
            new_corpus.append(sentence)
    except:
        print(sentence)

In [37]:
kor_corpus = new_corpus

* 음향모델에서 사용하는 2,040개의 문자 레이블로만 이루어진 문장 수 확인

In [38]:
acoustic_labels = list(pd.read_csv('train_labels.csv')['char'])
count = 0

for sentence in kor_corpus:
    for ch in sentence:
        if ch not in acoustic_labels:
            count += 1
            break

print(len(kor_corpus) - count)

2992120


* 조건에 만족하는 2,992,638개의 문장들로 최종 코퍼스 구성

In [41]:
final_corpus = list()

for sentence in kor_corpus:
    flag = True
    for ch in sentence:
        if ch not in acoustic_labels:
            flag = False
            break
    if flag:
        final_corpus.append(sentence)

len(final_corpus)

2992120

In [44]:
import csv

def load_label(label_path, encoding='utf-8'):
    char2id = dict()
    id2char = dict()

    with open(label_path, 'r', encoding=encoding) as f:
        labels = csv.reader(f, delimiter=',')
        next(labels)

        for row in labels:
            char2id[row[1]] = row[0]
            id2char[int(row[0])] = row[1]

    return char2id, id2char

char2id, id2char = load_label('train_labels.csv')

In [49]:
corpus_labels = list()

for sentence in final_corpus:
    label = str()
    for ch in sentence:
        if idx == 0:
            label += char2id[ch] + ' '
        elif idx == len(sentence) - 1:
            label += char2id[ch]
        else:
            label += char2id[ch] + ' '
    
    corpus_labels.append(label)
    
print(len(final_corpus))
print(len(corpus_labels))

2992120
2992120


In [50]:
corpus_dict = {'ko' : final_corpus,
               'id' : corpus_labels}
corpus_df = pd.DataFrame(corpus_dict)

corpus_df.head()

,ko,id
0,어디 보자...,8 190 0 42 45 1 1 1
1,칠대 왕국 종족 중에 오크는 처음 듣는다,318 50 0 576 170 0 363 401 0 129 17 0 57 238 4...
2,이번에 새로 개시할 제품 계열은 광고 모델을 계약해서 홍보하는 게 좋겠어요.,3 93 17 0 260 38 0 168 47 91 0 43 467 0 142 22...
3,라니스터 가의,32 20 79 162 0 6 130
4,별 희한한 생각이 다 떠오르곤 하죠,233 0 439 27 27 0 71 100 3 0 15 0 512 57 114 5...


In [51]:
len(corpus_df)

2992120

* 데이터셋 피클로 저장

In [52]:
import pickle

with open('corpus_df.bin', 'wb') as f:
    pickle.dump(corpus_df, f)